In [1]:
import numpy as np

from matplotlib import pyplot as plt

In [2]:
from nbodykit.source.catalog import ArrayCatalog
from nbodykit.lab import FFTPower

In [3]:
from abacusnbody.data.compaso_halo_catalog import CompaSOHaloCatalog

In [4]:
def measure_pkl(pos, uzs, Lbox=2000, nc=128, dk=0.005):

    hpos = 1.0 * pos
    hpos[:,2] += uzs

    data = {'Position':hpos}
    hcat = ArrayCatalog(data)
    hmesh = hcat.to_mesh(Nmesh=nc, BoxSize=Lbox, compensated=True, window='cic')

    r = FFTPower(hmesh, mode='2d',poles=[0,2,4],los=[0,0,1],Nmu=5, dk=dk)    
    k, Nk, p0k, p2k, p4k = r.poles['k'], r.poles['modes'], r.poles['power_0'].real, r.poles['power_2'].real, r.poles['power_4'].real

    sn = r.power.attrs['shotnoise']

    return k, p0k, p2k, p4k, sn, Nk, r

In [5]:
def measure_pnm(pos, uzs, Lbox=2000, nc=128, dk=0.005, max_order = 4,\
                for_cov=False, redshift_space=False):

    hpos = 1.0 * pos

    if redshift_space:
        hpos[:,2] += uzs
        
    data = {'Position':hpos}
    hcat = ArrayCatalog(data)
    
    meshes = []
    
    for ii in range(max_order+1):
        vel_name = 'u%d'%(ii)
        hcat[vel_name] = uzs**ii
        meshes += [hcat.to_mesh(Nmesh=nc, BoxSize=Lbox, compensated=True, window='cic', position='Position',value=vel_name),]

    ret = []

    ells = [0, 1, 2, 3, 4]
    
    for vel_order in range(max_order+1):

        #print("Now computing spectra with %d velocities"%(vel_order))
    
        counter = 0
        data = []
        #header = ''
        for ii in range(vel_order+1):
            jj = vel_order - ii
            
            if jj >= ii: # only distinct pairs
                #header += ' (%d,%d) |'%(ii,jj)
                rij = FFTPower(meshes[ii], mode='1d', poles=list(ells), second=meshes[jj], los=[0,0,1],dk=dk) 
            
                if counter == 0:
                    data += [rij.poles['k'],]
            
                if vel_order%2 == 0:
                    data += [ rij.poles['power_%d'%(ell)].real for ell in ells ]
                else:
                    data += [ rij.poles['power_%d'%(ell)].imag for ell in ells ]
            
                counter += 1
        
        ret += [data,]

    if for_cov:
        data = []
        
        ii, jj = 1, 2
        rij = FFTPower(meshes[ii], mode='1d', poles=list(ells), second=meshes[jj], los=[0,0,1],dk=dk)
        data += [rij.poles['k'],]
        data += [ rij.poles['power_%d'%(ell)].imag for ell in ells ]

        ii, jj = 2, 2
        rij = FFTPower(meshes[ii], mode='1d', poles=list(ells), second=meshes[jj], los=[0,0,1],dk=dk)
        data += [ rij.poles['power_%d'%(ell)].real for ell in ells ]

    ret += [data,]
    
    return ret

In [6]:
basedir = '/global/cfs/cdirs/desi/public/cosmosim/AbacusSummit/'

In [7]:
filebase = "pnms/"+"_".join(sample.split("_")[:3]) + '_' + sample.split("_")[-1][-6:] + '_' + f'{logMmin:.1f}' + '_' + f'{logMmax:.1f}'

NameError: name 'sample' is not defined

In [8]:
sample

NameError: name 'sample' is not defined

In [9]:
logMmin = 12.5
logMmax = 13.0
z = 0.3

In [13]:
p0ks, p2ks, p4ks = [], [], []
v1ks, v3ks = [], []
s0ks, s2ks = [], []


for ii in range(1,25):

    print(ii)
    
    sample = f'AbacusSummit_base_c000_ph{ii:03}/halos/z{z:.3f}'
    filename = basedir + sample

    cat = CompaSOHaloCatalog(basedir+sample,fields=('x_com','v_com','N'))
    
    particle_mass = cat.header['ParticleMassHMsun'] # / (cat.header['H0']/100)
    Ms = cat.halos['N'] * particle_mass
    mass_iis = (Ms > 10.**logMmin) * (Ms < 10**logMmax)
    cat.halos = cat.halos[mass_iis]

    Lbox = cat.header['BoxSize']
    nc = 256

    OmegaM = cat.header['Omega_M']
    z = cat.header['Redshift']
    Ez = np.sqrt(OmegaM * (1+z)**3 + 1 - OmegaM)
    vfac = 0.01 * (1 + z) / Ez # convert km/s to h^{-1} Mpc, u = v / (aH) = v * (1+z) / (100 * E(z))
    uz = cat.halos['v_com'][:,2] * vfac


    res = measure_pnm(cat.halos['x_com'], uz, Lbox=Lbox, nc=nc, dk=0.005, max_order=2,\
                      for_cov=True, redshift_space=True)

    kk, p0k, p1k, p2k, p3k, p4k = res[0]

    kk, p01_0, p01_1, p01_2, p01_3, p01_4 = res[1]

    kk, p02_0, p02_1, p02_2, p02_3, p02_4, p11_0, p11_1, p11_2, p11_3, p11_4 = res[2]

    kk, p12_0, p12_1, p12_2, p12_3, p12_4, p22_0, p22_1, p22_2, p22_3, p22_4 = res[3]

    p0ks += [p0k,]
    p2ks += [p2k,]
    p4ks += [p4k,]
    v1ks += [2j * p01_1,]
    v3ks += [2j * p01_3,]
    s0ks += [(2*p02_0 - 2*p11_0),]
    s2ks += [(2*p02_2 - 2*p11_2),]

    filebase = "pnms/"+"_".join(sample.split("_")[:3]) + '_' + sample.split("_")[-1][-6:] + '_' + f'{logMmin:.1f}' + '_' + f'{logMmax:.1f}'
    
    np.savetxt(filebase + f'_ph{ii:03}_pnms_v0_rsd.txt', np.array(res[0]).T)
    np.savetxt(filebase + f'_ph{ii:03}_pnms_v1_rsd.txt', np.array(res[1]).T)
    np.savetxt(filebase + f'_ph{ii:03}_pnms_v2_rsd.txt', np.array(res[2]).T)
    np.savetxt(filebase + f'_ph{ii:03}_pnms_higher_rsd.txt', np.array(res[3]).T)
    

1


/global/homes/s/sfschen/.conda/envs/nbodykit_env/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The window argument is deprecated. Use `resampler=` instead
  app.launch_new_instance()


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
